<a href="https://colab.research.google.com/github/kamgorny/neural_networks/blob/main/CNN_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries needed for the task:


In [ ]:
%tensorflow_version 2.x

import os
import numpy as np
import pandas as pd
import shutil

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers

import matplotlib.pyplot as plt

Download datasets:

In [ ]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ann-course/flying-vehicles.zip
!unzip -q flying-vehicles.zip

Prepare directories:

In [ ]:
!rm -rf ./images

base_dir = './data/planes'
data_dir = './images'

raw_no_of_files = {}
classes = ['drone', 'fighter-jet', 'helicopter', 'missile', 'passenger-plane', 'rocket']
# Iterate over classes and get the size of each dataset.
for dir in classes:
    raw_no_of_files[dir] = len(os.listdir(os.path.join(base_dir, dir)))

raw_no_of_files.items()


if not os.path.exists(data_dir):
    os.mkdir(data_dir)

# Training dataset
train_dir = os.path.join(data_dir, 'train')    
# Validation dataset
valid_dir = os.path.join(data_dir, 'valid')    
# Testing dataset
test_dir = os.path.join(data_dir, 'test')     

train_helicopter_dir = os.path.join(train_dir, 'helicopter')
train_missile_dir = os.path.join(train_dir, 'missile')

valid_helicopter_dir = os.path.join(valid_dir, 'helicopter')
valid_missile_dir = os.path.join(valid_dir, 'missile')

test_helicopter_dir = os.path.join(test_dir, 'helicopter')
test_missile_dir = os.path.join(test_dir, 'missile')

data_dirs = [train_helicopter_dir, train_missile_dir, valid_helicopter_dir, valid_missile_dir, test_helicopter_dir, test_missile_dir]

for directory in (train_dir, valid_dir, test_dir):
    if not os.path.exists(directory):
        os.mkdir(directory)

for dir in data_dirs:
    if not os.path.exists(dir):
        os.mkdir(dir)        

Filter dataset and exclude files with extensions other that *.jpg*, *.png* or *.jpeg*:

In [ ]:
helicopter_filenames = os.listdir(os.path.join(base_dir, 'helicopter'))
missile_filenames = os.listdir(os.path.join(base_dir, 'missile'))

helicopter_filenames = [filename for filename in helicopter_filenames if filename.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]
missile_filenames = [filename for filename in missile_filenames if filename.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]

Calculate index of last file for each dataset:

In [ ]:
size = min(len(helicopter_filenames), len(missile_filenames))

train_size = int(np.floor(0.7 * size))
valid_size = int(np.floor(0.2 * size))
test_size = size - train_size - valid_size

train_idx = train_size
valid_idx = train_size + valid_size
test_idx = train_size + valid_size + test_size


Divide files between train, validation and test datasets:

In [ ]:
for i, fname in enumerate(helicopter_filenames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'helicopter', fname)
        dst = os.path.join(train_helicopter_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'helicopter', fname)
        dst = os.path.join(valid_helicopter_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'helicopter', fname)
        dst = os.path.join(test_helicopter_dir, fname)
        shutil.copyfile(src, dst)

for i, fname in enumerate(missile_filenames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'missile', fname)
        dst = os.path.join(train_missile_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'missile', fname)
        dst = os.path.join(valid_missile_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'missile', fname)
        dst = os.path.join(test_missile_dir, fname)
        shutil.copyfile(src, dst) 

print('Helicopter - training set size: ', len(os.listdir(train_helicopter_dir)))
print('Helicopter - validation set size: ', len(os.listdir(valid_helicopter_dir)))
print('Helicopter - test set size: ', len(os.listdir(test_helicopter_dir)))

print('Missile - training set size: ', len(os.listdir(train_missile_dir)))
print('Missile - validation set size: ', len(os.listdir(valid_missile_dir)))
print('Missile - test set size: ', len(os.listdir(test_missile_dir)))             

In [ ]:
#@title Pick and index of an helicopter image from the training dataset:

idx = 380 #@param {type:'slider', min:0, max:919}
names_mapping = dict(enumerate(helicopter_filenames))
img_path = os.path.join(train_helicopter_dir, names_mapping[idx])

img = image.load_img(img_path)

plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.grid(False)
plt.axis(False)

In [ ]:
#@title Pick and index of an missile image from the training dataset:

idx = 239 #@param {type:'slider', min:0, max:919}
names_mapping = dict(enumerate(missile_filenames))
img_path = os.path.join(train_missile_dir, names_mapping[idx])

img = image.load_img(img_path)

plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.grid(False)
plt.axis(False)

Data augmentation:

In [ ]:
training_data_generator = ImageDataGenerator(
    rotation_range=40,     # the range of random rotation of an image
    rescale=1./255.,
    width_shift_range=0.2,  # the range of horizontal shift of an image
    height_shift_range=0.2, # the range of vertical shift of an image
    shear_range=0.2,        # the range of random shear of an image
    zoom_range=0.2,         # the range of random zoom of an image
    horizontal_flip=True,   # should the image be flipped horizontally
    fill_mode='nearest'     # how to deal with new pixels created during the image transformation
)

# rescale all the images by 1/255 bias.
validation_data_generator = ImageDataGenerator(rescale=1./255.)

train_generator = training_data_generator.flow_from_directory(directory=train_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='binary')

valid_generator = validation_data_generator.flow_from_directory(directory=valid_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='binary')

In [ ]:
def display_augmented_images(directory, idx):
    """
    The function returns the plot with example images
    created with the data augmentation technique.
    """
    file_names = [os.path.join(directory, file_name) for file_name in os.listdir(directory)]
    img_path = file_names[idx]
    img = image.load_img(img_path, target_size=(150, 150))

    x = image.img_to_array(img)
    x = np.expand_dims(x, 0)
    # we can also use x = x.reshape((1, ) + x.shape)
    print(x.shape)
    i = 1
    plt.figure(figsize=(16, 8))
    for batch in training_data_generator.flow(x, batch_size=1):
        plt.subplot(3, 4, i)
        plt.grid(False)
        imgplot = plt.imshow(image.array_to_img(batch[0]))
        i += 1
        if i % 13 == 0:
            break

In [ ]:
#@title Helicopter dataset:
idx = 467 #@param {type:'slider', min:0, max:919}            
display_augmented_images(train_helicopter_dir, idx)

Building model:

In [ ]:
model = Sequential()
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(units=1024, activation='relu'))
model.add(layers.Dense(units=1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer=optimizers.RMSprop(learning_rate=0.01),
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [ ]:
batch_size = 32
steps_per_epoch = train_size // batch_size
validation_steps = valid_size // batch_size

history = model.fit_generator(generator=train_generator,
                             steps_per_epoch=steps_per_epoch,
                             epochs=30,    # 100
                             validation_data=valid_generator,
                             validation_steps=validation_steps)